In [1]:
from sklearn import preprocessing
from sklearn.decomposition import PCA 
import numpy as np
import pandas as pd 
import math
import time 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

np.random.seed(10)

# Data loading
df = pd.read_csv('dataset\\LP1\\LP1_DP.csv')

ylabels = df.Class.unique()
output_dim = len(ylabels)
full_data = df.to_numpy()
full_data = np.transpose(np.transpose(full_data)[1:])

In [2]:
train_data = []
lp_tmp = []
np.random.shuffle(full_data)
for tp in full_data:
    train_data.append(tp[1:])
    lp_tmp.append(tp[0])

# oneHot Encoding
train_label = np.zeros( ( len(train_data), output_dim ) )
for i in range(len(lp_tmp)):
    train_label[i][lp_tmp[i]-1] = 1     # let 1-5 turns to 0-4

In [3]:
#data preprocessing
x_scale = preprocessing.scale(train_data)
x_normalized = preprocessing.normalize(x_scale, norm='l2')
pca=PCA(whiten=True)
# newData=pca.fit_transform(x_normalized)
newData = x_normalized
np.shape(newData)

(88, 90)

In [4]:
def initialize(Layer):
    Weights = []
    Bias = []
    for i in range ( 1, len(Layer) ):
        weight = np.random.randn( Layer[i-1], Layer[i]) * np.sqrt(2 / Layer[i])
        b = np.random.randn(Layer[i], 1) * np.sqrt(2 / Layer[i])
        Weights.append(weight)
        Bias.append(b)

    # to make index of layer 1 from 0 to 1
    Weights.insert(0, [])
    Bias.insert(0, [])
    Weights = np.asarray(Weights)
    Bias = np.asarray(Bias)

    return Weights, Bias

def Minmax_scale(w):
    return np.array( (w-w.min()) / (w.max()-w.min()) )
minmax = 0
def Partition(data, label, ratio):
    if minmax == 1 :
        data = np.transpose(data).astype(np.float64)
        for i in range (len(data) ):
            data[i] = Minmax_scale(data[i])
        data = np.transpose(data)
    t_data = data[0:int(len(data)*ratio) ]
    t_label = label[0:int(len(label)*ratio) ]
    verify_data = data[int(len(data)*ratio):len(data)]
    verify_label = label[int(len(data)*ratio):len(data)]
    return t_data, t_label, verify_data, verify_label

print("D")

def Sigmoid( n ):
    l = []
    for tmp in n:
        if np.sum(tmp) >= 0:
            l.append( 1 / (1 + math.exp(-tmp) ) )
        else:
            l.append( math.exp(tmp) / ( 1 + math.exp(tmp) )  )
    return l

# def sigmoid(data):
#   if np.sum(data) >= 0:
#     return  1 / ( 1 + math.exp(-np.sum(data)))
#   else:
#     return math.exp(np.sum(data)) / (1 + math.exp(np.sum(data)) )

def Binary_crossEntropy(y, a):
    CE = 0
    for i in range( len(y) ):
        CE += y[i] * math.log(a[i] + 1e-15 ) + (1 - y[i]) * math.log(1 - a[i] + 1e-15)
    return CE
def FeedForward(data, Weight, bias):
    n = np.matmul( data, Weight) + bias.transpose() 
    return np.asarray( Sigmoid(n[0]) ) 

def Prediction(a):
    index = 0
    for i in range( len(a) ):
        if a[i] > a[index]:
            index = i
    return index

D


In [5]:
train_ratio = 0.8
Layer = [40, 20, 10, 5]

Layer.insert(0, len(newData[0]))
Layer.append(output_dim)

Weights, Bias = initialize(Layer)
t_data, t_label, verify_data, verify_label = Partition(newData, train_label, train_ratio)

C:\Python39\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [6]:
learning_rate = 0.01
Epoch = 501
tau = 0.001

end_flag = False
last_acc_T = 0
last_acc_V = 0
ce = []
writee = False
strr = ""
for ep in range(Epoch):
    totalLoss = 0
    L = len(Layer)-1
    acc_t = 0
    acc_v = 0
    flag = (ep % 10 == 0)
    string = ""
    for i in range(len(t_data)):
        a = []
        a.append( t_data[i] )
        for l in range(1, L+1 ):
            a.append(FeedForward(a[l-1],  Weights[l], Bias[l]) )
        
        # BackWard
        Delta = [ a[L] - t_label[i] ]
        for l in range(L-1, 0, -1):
            Delta.insert(0, np.matmul( Delta[0], Weights[l+1].transpose() ) * ( a[l]*(1-a[l])  )  )
        Delta.insert(0, [])
        
        # update Weights and Bias
        for l in range(1, L+1):
            t = np.matmul( Delta[l][np.newaxis].T, a[l-1][np.newaxis] )
            Weights[l] = Weights[l] - learning_rate * t.T
            Bias[l] = Bias[l] - learning_rate * Delta[l]
        
        # Calculate loss and training accuracy
        if flag : 
            totalLoss -= Binary_crossEntropy( t_label[i], a[L] )
            p = Prediction(a[L])
            if t_label[i][p] == 1:
                acc_t += 1

    # Calculate verify accuracy
    if flag : 
        for i in range( len(verify_data) ):
            out = verify_data[i]
            for l in range(1, L+1 ):
                out = FeedForward(out, Weights[l], Bias[l])
            p = Prediction(out)
            if verify_label[i][p] == 1:
                acc_v += 1

        if last_acc_T < acc_t / len(t_data) and last_acc_V > acc_v / len(verify_data):
            print("< Overtraining leads to overfitting >" )
            end_flag = True
        last_acc_T = acc_t
        last_acc_V = acc_v

    if (ep+1) == Epoch:
        print("< Reach maximum Epoch >" )
        end_flag = True

    if flag and totalLoss / len(t_data) < tau:
        print("< Loss is low enough ( tau: " + str(tau) + ") >"  )
        end_flag = True

    if flag:
        print("<<< Status" + " " + strr + " >>>")
        print("Number of train_data: " + str( len(t_data)) )
        print("Number of verify_data: " + str( len(verify_data)) )
        print("Number of Hidden Layer: " + str( len(Layer)-2) )
        print("Number of Neuron in each Hidden Layer: " + str(Layer[1:len(Layer)-1]) )
        print("Learning Rate: " + str(learning_rate) )
        print("Epoch: " + str(ep+1) )
        print("Loss: " + str( totalLoss / len(t_data) ))
        print("Train accuracy: " + str(acc_t / len(t_data)) )
        print("Verify accuracy: " + str(acc_v / len(verify_data)) )
        
    if writee :    
        with open('Results\\' + strr + ' ' + 'Result '+time.strftime('%Y-%m-%d %H-%M-%S', time.localtime()) +  '.txt', mode = 'w') as text_file:
            string += "<<<Status>>>\n"
            string += "Number of train_data: " + str( len(t_data)) +"\n"
            string += "Number of verify_data: " + str( len(verify_data)) +"\n"
            string += "Number of Hidden Layer: " + str( len(Layer)-2) +"\n"
            string += "Number of Neuron in each Hidden Layer: " + str(Layer[1:len(Layer)-1]) +"\n"
            string += "Learning Rate: " + str(learning_rate) +"\n"
            string += "Epoch: " + str(ep+1) +"\n"
            string += "Loss: " + str( totalLoss / len(t_data) )+"\n"
            string += "Train accuracy: " + str(acc_t / len(t_data)) +"\n"
            string += "Verify accuracy: " + str(acc_v / len(verify_data)) +"\n"
            text_file.write(string)
            text_file.close()

    if end_flag:
        break

<<< Status  >>>
Number of train_data: 70
Number of verify_data: 18
Number of Hidden Layer: 4
Number of Neuron in each Hidden Layer: [40, 20, 10, 5]
Learning Rate: 0.01
Epoch: 1
Loss: 3.4916230740268794
Train accuracy: 0.15714285714285714
Verify accuracy: 0.3333333333333333
<<< Status  >>>
Number of train_data: 70
Number of verify_data: 18
Number of Hidden Layer: 4
Number of Neuron in each Hidden Layer: [40, 20, 10, 5]
Learning Rate: 0.01
Epoch: 11
Loss: 2.1652735069953124
Train accuracy: 0.4142857142857143
Verify accuracy: 0.2777777777777778
<<< Status  >>>
Number of train_data: 70
Number of verify_data: 18
Number of Hidden Layer: 4
Number of Neuron in each Hidden Layer: [40, 20, 10, 5]
Learning Rate: 0.01
Epoch: 21
Loss: 2.1609668770740447
Train accuracy: 0.4142857142857143
Verify accuracy: 0.2777777777777778
<<< Status  >>>
Number of train_data: 70
Number of verify_data: 18
Number of Hidden Layer: 4
Number of Neuron in each Hidden Layer: [40, 20, 10, 5]
Learning Rate: 0.01
Epoch: 31


In [56]:
torch.manual_seed(1)
class MLP(nn.Module):
    def __init__(self, input, output):
        super(MLP,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(input, 64),
            nn.BatchNorm1d(64),
        )
        self.layer2 = nn.Sequential(
            nn.Linear(64, 32),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(32, 16),
        )
        self.layer4 = nn.Sequential(
            nn.Linear(16, 8),
        )
        self.layer5 = nn.Sequential(
            nn.Linear(8, output),
            nn.Softmax(dim=1)
        )
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        return x
model = MLP(len(t_data[0]), output_dim)
criterion = nn.MultiLabelSoftMarginLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
Epoch = 6400

maxt = 0
maxv = 0
minl = 10000
for epoch in range(Epoch):
    acc_t = 0
    acc_v = 0
    mean_loss = 0
    tmp1, tmp2, tmp3 = maxt, maxv, minl
    optimizer.zero_grad()
    inputs = torch.autograd.Variable( torch.FloatTensor(t_data))
    outputs = model(inputs)
    loss = criterion( outputs, torch.tensor(t_label) )
    mean_loss += loss.item()
    loss.backward()
    optimizer.step()

    minl = min(minl, mean_loss/len(t_data))
    for i in range (len(outputs)):
        p = Prediction(outputs[i])
        if t_label[i][p] == 1:
            acc_t += 1
    var_in = torch.autograd.Variable( torch.FloatTensor(verify_data))
    var_out = model(var_in)
    for i in range (len(var_out)):
        p = Prediction(var_out[i])
        if verify_label[i][p] == 1:
            acc_v += 1
    maxt = max(maxt, acc_t / len(t_data))
    maxv = max(maxv, acc_v / len(verify_data))
    if tmp1 != maxt or tmp2 != maxv or tmp3 != minl :
        print("Epoch:" + str(epoch))
        print( mean_loss/len(t_data) )
        print("training acc: " + str(acc_t / len(t_data)))
        print("testing acc: " + str(acc_v / len(verify_data)))

    # if epoch % 100 == 0:
    #     print("Epoch:" + str(epoch))
    #     print( mean_loss/len(t_data) )
    #     for i in range (len(outputs)):
    #         p = Prediction(outputs[i])
    #         if t_label[i][p] == 1:
    #             acc_t += 1
    #     print("acc: " + str(acc_t / len(t_data)))

    #     var_in = torch.autograd.Variable( torch.FloatTensor(verify_data))
    #     var_out = model(var_in)
    #     for i in range (len(var_out)):
    #         p = Prediction(var_out[i])
    #         if verify_label[i][p] == 1:
    #             acc_v += 1
    #     print("testing acc: " + str(acc_v / len(verify_data)))

Epoch:0
0.010909966552744108
training acc: 0.2
testing acc: 0.2777777777777778
Epoch:1
0.010731913070289456
training acc: 0.5857142857142857
testing acc: 0.2777777777777778
Epoch:2
0.010477960409254444
training acc: 0.5571428571428572
testing acc: 0.2777777777777778
Epoch:3
0.010134855505763268
training acc: 0.5428571428571428
testing acc: 0.2777777777777778
Epoch:4
0.009835112960058817
training acc: 0.5285714285714286
testing acc: 0.5555555555555556
Epoch:5
0.009614541091176928
training acc: 0.7142857142857143
testing acc: 0.5555555555555556
Epoch:6
0.009540217211660073
training acc: 0.7
testing acc: 0.5555555555555556
Epoch:7
0.00950602626161916
training acc: 0.7
testing acc: 0.5555555555555556
Epoch:8
0.00943023688939153
training acc: 0.7428571428571429
testing acc: 0.5555555555555556
Epoch:9
0.00934252095161652
training acc: 0.7714285714285715
testing acc: 0.5
Epoch:10
0.009282838337275447
training acc: 0.8
testing acc: 0.5
Epoch:11
0.009213058635288355
training acc: 0.814285714285

In [8]:
loss

tensor(0.6017, dtype=torch.float64, grad_fn=<MeanBackward0>)